In [ ]:
#!pip install selenium

In [31]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

from bs4 import BeautifulSoup
import requests

In [32]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')
service = Service("./chromedriver")  
driver = webdriver.Chrome(service=service, options=chrome_options)

In [33]:
QUERY = "I-MOD"          
START_DATE = "2022.07.01" 
END_DATE = "2023.06.30"
START_PAGE = 1            
END_PAGE = 8
article_filename = "article_" + QUERY + "_" + START_DATE + "_" + END_DATE + "_" + str(START_PAGE) + ".txt"

In [40]:
f = open(article_filename, "w", encoding="utf-8")
news_count = 0

for page in range(START_PAGE, END_PAGE*10+1, 10):
    URL = "https://search.naver.com/search.naver?&where=news&query=" + QUERY 
    URL += "&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds="
    URL += START_DATE + "&de=" + END_DATE + "&docid=&&start=" + str(page) + "&refresh_start=0"
    driver.get(URL)
    time.sleep(2)
    
    try:
        news_list = driver.find_element(By.CLASS_NAME, "list_news").find_elements(By.CLASS_NAME, "bx")
    except:
        break
    
    
    for news in news_list[:]:
        link_list = news.find_element(By.CLASS_NAME, "info_group").find_elements(By.TAG_NAME, "a")
        if len(link_list) == 1:
            continue
        article_url = link_list[1].get_attribute("href").strip()
        link_list[1].click()
        time.sleep(1)
        current_window = driver.current_window_handle
        try:
            new_window = [window for window in driver.window_handles if window != current_window][0]
            driver.switch_to.window(new_window)
        except:
            driver.switch_to.window(current_window)
            continue
        time.sleep(2) 
        
        try:               
            if "sports.news" in article_url:
                source_label = driver.find_element(By.ID, "pressLogo").find_element(By.TAG_NAME, "img")
                source = source_label.get_attribute("alt").strip()
                datetime = driver.find_element(By.CLASS_NAME, "info").find_element(By.TAG_NAME, "span").text.replace("기사입력", "").strip()
                content = driver.find_element(By.CLASS_NAME, "news_end").text.strip().replace("\n", " ")
                title = driver.find_element(By.CLASS_NAME, "title").text.strip()
                
            elif "n.news" in article_url:
                try:
                    try:
                        source_label = driver.find_element(By.ID, "pressLogo").find_element(By.TAG_NAME, "img")
                        source = source_label.get_attribute("alt").strip()
                    except:
                        source_label = driver.find_element(By.CLASS_NAME, "media_end_head_top_logo").find_element(By.TAG_NAME, "img")
                        source = source_label.get_attribute("alt").strip()
                    datetime = driver.find_element(By.CLASS_NAME, "_ARTICLE_DATE_TIME").text.strip()            
                    content = driver.find_element(By.CLASS_NAME, "_article_content").text.strip().replace("\n", " ")
                    title = driver.find_element(By.CLASS_NAME, "media_end_head_headline").text.strip()
                except:
                    source_label = driver.find_element(By.CLASS_NAME, "press_logo")
                    source_img = source_label.find_element(By.TAG_NAME, "img")
                    source = source_img.get_attribute("alt").strip()
                    datetime = driver.find_element(By.CLASS_NAME, "author")
                    datetime = datetime.find_element(By.TAG_NAME, "em").text.strip()
                    content = driver.find_element(By.ID, "articeBody").text.strip().replace("\n", " ")
                    title = driver.find_element(By.CLASS_NAME, "end_tit").text.strip()
                    
            else:
                driver.close()
                time.sleep(2)
                driver.switch_to.window(current_window)
                continue
                
        except:
            driver.close()
            time.sleep(2)
            driver.switch_to.window(current_window)
            f.flush()
            continue
            
        f.write(source + "\t" + datetime + "\t" + 
                     article_url + "\t" + title + "\t" + content + "\n")
        
        driver.close()
        time.sleep(2)
        driver.switch_to.window(current_window)
        f.flush()
        
f.close()

print()
print("* 최대", news_count, "개 기사 수집이 완료되었습니다.")
print("* 수집된 기사는 아래 파일에 저장되었습니다.")
print("  :", article_filename)


* 최대 0 개 기사 수집이 완료되었습니다.
* 수집된 기사는 아래 파일에 저장되었습니다.
  : article_I-MOD_2022.07.01_2023.06.30_1.txt


In [41]:
driver.close()